In [1]:
"""
Name: example_analysis.ipynb
Authors: Stephan Meighen-Berger
Example how to construct a frequency analysis
"""

'\nName: example_analysis.ipynb\nAuthors: Stephan Meighen-Berger\nExample how to construct a frequency analysis\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import imageio
from tqdm import tqdm

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day, config

In [6]:
# Some example settings
config['scenario']['population size'] = 100
config['scenario']['duration'] = 6000
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 16
config['scenario']['injection']['y range'] = [0., 10.]
config['scenario']['light prop'] = {
            "switch": True,
            "x_pos": 5.,
            "y_pos": 10.,
        }
config['scenario']['detector'] = {
    "switch": True,
    "type": "PMTSpec",
    "response": True,
    "acceptance": "Flat",
    "mean detection prob": 0.5
}
# Organisms 
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 2.
config['organisms']['photon yield'] = 1e10
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    'offset': None,
}
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    "offset": np.array([0., 0.]),
}
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.15,
    "x_pos": 2.,
    "y_pos": 5.,
}
# Water
config['water']['model']['name'] = 'custom' # 'potential cylinder'
config['water']['model']['directory'] = "../data/current/long_run/"
config['water']['model']['time step'] = 0.1
config['advanced']['starting step'] = 0

In [7]:
# Creating a fourth_day object
fd = Fourth_Day()

In [ ]:
# Launching solver
fd.sim()

In [ ]:
# from scipy.stats import gamma
# test_array=np.full((24,),10)
# gamma.pdf(test_array,a=3)

In [ ]:
# Plotting standards
std_size = 3.
fontsize = 10.
lw=1.
h_length=0.2

In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
from itertools import groupby
def scramble_data(data, toprint=False):
    peaks = np.array([list(v) for k,v in groupby(data, key = lambda x: x != 0) if k != 0])
    number_of_peaks = len(peaks)
    number_of_elements = np.sum(np.array([len(it) for it in peaks]))
    total_number_of_elements = data.shape[0]
    number_of_pauses = total_number_of_elements - number_of_elements
    peak_order = np.arange(len(peaks))
    np.random.shuffle(peak_order)
    if toprint:
        print(number_of_pauses)
        print(number_of_peaks)
    peak_pos = np.random.choice(number_of_pauses, size=number_of_peaks, replace=False)
    peak_pos = np.sort(peak_pos)
    peak_diffs = np.diff(peak_pos)
    zero_arrs = np.array([
        np.zeros(peak_diff)
        for peak_diff in peak_diffs
    ])
    # Need to add 0
    to_add = np.zeros(number_of_pauses - np.sum(peak_diffs))
    list_form = zero_arrs.tolist()
    list_form.append(to_add)
    zero_arrs = np.array(list_form)
    splitting_base = np.array([
        np.concatenate((zero_arrs[i], peaks[i]))
        for i in peak_order
    ])
    new_data = np.array([item for sublist in splitting_base for item in sublist])
    return new_data

In [ ]:
wavelengths = {
    "Detector 1": ["1", "#4575b4"],
    "Detector 5": ["2", "#91bfdb"],
    "Detector 8": ["3", "#e0f3f8"],
    "Detector 3": ["4", "#fee090"],
    "Detector 10": ["5", "#fc8d59"],
}

In [ ]:
# Totals
total = fd.measured["Detector 1"].values
for detector in wavelengths.keys():
    if detector == "Detector 1":
        continue
    fd.measured[detector].values
    total += fd.measured[detector].values

In [ ]:
# Scrambling
scrambled_example = scramble_data(total, toprint=False)

In [ ]:
# Scambling multiple times
num_scrambles = 1000
scrambled_array = np.array([
    scramble_data(total, toprint=False)
    for i in range(num_scrambles)
])
fft_det = np.fft.fft(scrambled_array, norm='ortho', axis=1)
fft_freq = np.fft.fftfreq(scrambled_array[0].size, d=0.1)
# The mean and std
mean_arr = np.mean(np.abs(fft_det.real) / 1e7, axis=0, dtype=np.float64)
std_arr = np.std(np.abs(fft_det.real) / 1e7, axis=0, dtype=np.float64)

In [ ]:
# Energy distributions
# startpoint = 240.
# width = 35
wavelengths = {
    "Detector 1": ["1", "#4575b4"],
    "Detector 5": ["2", "#91bfdb"],
    "Detector 8": ["3", "#e0f3f8"],
    "Detector 3": ["4", "#fee090"],
    "Detector 10": ["5", "#fc8d59"],
#     "Detector 2": ["6", "#d73027"],
}
figure, (ax1) = plt.subplots(1, 1, figsize=(std_size, std_size * 6. / 8.), sharex=True)
fft_total = np.fft.fft(total, norm='ortho')
fft_total_freq = np.fft.fftfreq(total.size, d=0.1)
cut_off = np.argmax(fft_freq)
# ax1.plot(fft_freq[:cut_off],
#          smooth(np.abs(fft_det.real[:cut_off]), 1),
#          lw=lw, color='k')
# for scambled in fft_det:
#     ax1.plot(fft_freq[:cut_off],
#              smooth(np.abs(scambled.real[:cut_off]), 21) / 1e7,
#              lw=lw, color='b', alpha=0.2)
ax1.fill_between(fft_freq[:cut_off],
                 mean_arr.real[:cut_off] - 2. * std_arr[:cut_off] * mean_arr.real[:cut_off],
                 mean_arr.real[:cut_off] + 2. * std_arr[:cut_off] * mean_arr.real[:cut_off],
                 color='#009C3B', alpha=1.)
ax1.fill_between(fft_freq[:cut_off],
                 mean_arr.real[:cut_off] - std_arr[:cut_off] * mean_arr.real[:cut_off],
                 mean_arr.real[:cut_off] + std_arr[:cut_off] * mean_arr.real[:cut_off],
                 color='#FFDF00', alpha=1.)
ax1.plot(fft_freq[:cut_off],
         mean_arr.real[:cut_off],
         lw=lw, color='r')
ax1.plot(fft_total_freq[:cut_off],
         smooth(np.abs(fft_total.real[:cut_off]), 21) / 1e7,
         lw=lw, color='k')
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$\nu\;[\mathrm{Hz}]$', fontsize=fontsize)
ax1.set_ylabel(r'$|\mathrm{FFT}(N_\gamma)| / 1e7$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
# ax1.grid(True)
ax1.set_ylim(0.,1.5)
ax1.set_xlim(5e-2, 1.)
plt.tight_layout()
plt.show()

In [ ]:
figure.savefig(PICS + "MC_FFT.pdf",
               bbox_inches='tight')